## Homework 4: Dashboard Visualization
### Jenny Zhuo

The link of my blog is https://jennyzz17.github.io/  
On the webpage, please click on the "Life after Graduate School" tag in the right "Tags" section to see the blog for this assignment.  
The Github repository of this Github page is https://github.com/jennyzz17/jennyzz17.github.io  
Thank you so much!

In [1]:
import pandas as pd
import numpy as np
year_field = pd.read_excel("sed17-sr-tab011.xlsx",
                            skiprows = [0, 1],
                            header = [1])

In [2]:
year_life = year_field.iloc[6:10]
year_life = year_life.melt(
    id_vars = ["Field and Carnegie category"])
year_life.columns = ["Category", "Year", "Life"]

In [3]:
year_physcial = year_field.iloc[11:15]
year_physcial = year_physcial.melt(
    id_vars = ["Field and Carnegie category"])
year_physcial.columns = ["Category", "Year", "Physical"]

In [4]:
year_math = year_field.iloc[16:20]
year_math = year_math.melt(
    id_vars = ["Field and Carnegie category"])
year_math.columns = ["Category", "Year", "Mathematics"]

In [5]:
year_psy = year_field.iloc[21:25]
year_psy = year_psy.melt(
    id_vars = ["Field and Carnegie category"])
year_psy.columns = ["Category", "Year", "Psychology"]

In [6]:
year_engineer = year_field.iloc[26:30]
year_engineer = year_engineer.melt(
    id_vars = ["Field and Carnegie category"])
year_engineer.columns = ["Category", "Year", "Engineering"]

In [7]:
year_edu = year_field.iloc[31:35]
year_edu = year_edu.melt(
    id_vars = ["Field and Carnegie category"])
year_edu.columns = ["Category", "Year", "Education"]

In [8]:
year_human = year_field.iloc[36:40]
year_human = year_human.melt(
    id_vars = ["Field and Carnegie category"])
year_human.columns = ["Category", "Year", "Humanities"]

In [9]:
year_other = year_field.iloc[41:45]
year_other = year_other.melt(
    id_vars = ["Field and Carnegie category"])
year_other.columns = ["Category", "Year", "Other"]

In [10]:
year_field_plot = pd.concat([year_life,
                             year_physcial[["Physical"]],
                             year_math[["Mathematics"]],
                             year_psy[["Psychology"]],
                             year_engineer[["Engineering"]],
                             year_edu[["Education"]],
                             year_human[["Humanities"]],
                             year_other[["Other"]]],axis = 1)


In [11]:
gender_field = pd.read_excel("sed17-sr-tab015.xlsx",
                            skiprows = [0, 1],
                            header = [1])
other_index = gender_field[
    gender_field['Sex and major field of study'] == "Otherb"].index
gender_field.at[other_index, 'Sex and major field of study'] = "Other"
male_index = np.where(gender_field['Sex and major field of study'] == "Male")
female_index = np.where(gender_field['Sex and major field of study'] == "Female")
male_data = gender_field[45:90]
female_data = gender_field[90:]
male_field = male_data[male_data["Sex and major field of study"].isin 
                          (["Life sciences",
                           "Physical Sciences and Earth Sciences",
                           "Mathematics and Computer Sciences",
                           "Psychology and Social Sciences",
                           "Engineering",
                           "Education",
                           "Humanities and Arts",
                           "Other"])]
male_field = male_field.rename({"Sex and major field of study": "field"},axis = 1)
male_plot = male_field.drop("% change 2008–17",1).pivot_table(
    columns = ["field"]
)
male_plot["Year"] = male_plot.index
male_plot.columns = ["Education", 
                     "Engineering", "Life", 
                     "Other", "Year"]
male_plot["Gender"] = "Male"
female_field = female_data[female_data["Sex and major field of study"].isin
                          (["Life sciences",
                           "Physical Sciences and Earth Sciences",
                           "Mathematics and Computer Sciences",
                           "Psychology and Social Sciences",
                           "Engineering",
                           "Education",
                           "Humanities and Arts",
                           "Other"])]
female_field = female_field.rename({"Sex and major field of study": "field"},axis = 1)
female_plot = female_field.drop("% change 2008–17",1).pivot_table(
    columns = ["field"]
)
female_plot["Year"] = female_plot.index
female_plot.columns = ["Education", 
                     "Engineering", "Life", 
                     "Other", "Year"]
female_plot["Gender"] = "Female"
gender_plot = pd.concat([
    male_plot,
    female_plot
])


In [12]:
import pandas as pd
pd.options.mode.chained_assignment = None
input_data = pd.read_excel("sed17-sr-tab007.xlsx", 
                          skiprows = [0,1,2,3],
                          header = [1])
data_cleaned = input_data.set_index(input_data.columns[0])
plot_data = data_cleaned.rename(
    {"Unnamed: 1": "All fields"},
    axis = 1)
plot_data.index.names = ["stname"]

In [13]:
state_names = pd.read_csv(
    "https://gist.githubusercontent.com/dantonnoriega/bf1acd2290e15b91e6710b6fd3be0a53/raw/11d15233327c8080c9646c7e1f23052659db251d/us-state-ansi-fips.csv")
state_data = plot_data.merge(state_names, 
                             how = "inner",
                             on = "stname")
state_plot = state_data.filter(regex = "Total")
state_plot_re = state_plot.rename(columns = {"Total": "Life",
                            "Total.1": "Physical",
                            "Total.2": "Mathematics",
                            "Total.3": "Psychology",
                            "Total.4": "Engineering"})

state_plot_re2 = pd.concat([state_plot_re,
                            state_data[["stname"]],
                            state_data[[" stusps"]]
                           ], axis = 1)
state_plot_final = state_plot_re2.rename({" stusps": "Code"}, axis = 1)
state_plot_final["Code"] = state_plot_final["Code"].str.lstrip()


In [ ]:
import dash
from dash import dcc
import dash_bootstrap_components as dbc
from dash import html
from dash.dependencies import Input, Output
from dash.exceptions import PreventUpdate
import plotly.graph_objects as go
import plotly.express as px

field_list = gender_plot.columns[:4].tolist()
field_full = state_plot_re.columns[:-1].tolist()
years = sorted(year_field_plot['Year'].unique().tolist())
drop_down_trans = {
    "Life": "Life Sciences",
    "Engineering": "Engineering",
    "Education": "Education",
    "Other": "Other"
}
drop_down_second = {
    "Life": "Life Sciences",
    "Physical": "Physical Sciences and Earth Sciences",
    "Mathematics": "Mathematics and Computer Sciences",
    "Psychology": "Psychology and Social Sciences",
    "Engineering": "Engineering"
}

app = dash.Dash(__name__, external_stylesheets = [dbc.themes.FLATLY])
server = app.server

app.layout = html.H1(children = 
                     "Dashboard for Visualizing the Number of Doctorates Awarded")
app.layout = html.Div([
    dbc.Row([
        html.Br(), html.Br(),
        dbc.Col(lg = 1),
        dbc.Col([
            html.Br(),
            dbc.Label('Select a Field:'),
            dcc.Dropdown(id = 'field_dropdown_top',
                         placeholder = 'Select the Field',
                         value = 'Life',
                         options = [{'label': v, 'value': k}
                                  for k, v in drop_down_trans.items()])
        ], lg = 5),

        dbc.Col([
            html.Br(),
            dbc.Label('Select a Year:'),
            dcc.Slider(id = 'year_slider',
                       tooltip = {'always_visible': True},
                       min = min(years),
                       max = max(years),
                       value = 2008,
                       included = False,
                       step = None,
                       marks = {year: {'label': str(year)}
                              for year in years})
        ], lg = 5),
    ]),
    dbc.Row([
        dbc.Col([
            dcc.Graph(id = 'bar_gender',
                      config = {'displayModeBar': 'hover',
                              'displaylogo': False,
                              'showTips': True,
                              'modeBarButtonsToRemove': ['lasso2d', 'select2d', 'hoverClosestGeo'],
                              'toImageButtonOptions': {'format': 'svg'}})
        ], lg = 5),
        dbc.Col([
            dcc.Graph(id = 'bar_education',
                      config = {'displayModeBar': 'hover',
                               'displaylogo': False,
                              'showTips': True,
                              'modeBarButtonsToRemove': ['lasso2d', 'select2d', 'hoverClosestGeo'],
                              'toImageButtonOptions': {'format': 'svg'}})
        ], lg = 5)
    ]),
    dbc.Row([
        dbc.Col([], lg = 1),
        dbc.Col([
            dbc.Label('Select a Field:'),
            dcc.Dropdown(id = 'us-dropdown',
                         placeholder = "Select the Field",
                         value = 'Life',
                         options = [{'label': v, 'value': k}
                                  for k,v in drop_down_second.items()])

        ], lg = 6),
    ]),
    dbc.Row([
        dbc.Col([], lg = 10),
        dbc.Col([
            dcc.Graph(id = 'us-map', config = {'displayModeBar': False})
        ], lg = 10)
    ])
], style = {'backgroundColor': 'rgb(0,0,0,0)'})

# the bottom picture
@app.callback(Output('us-map', 'figure'),
              [Input('us-dropdown', 'value')])
def plot_us_map(field):
    if not field:
        raise PreventUpdate
    fig = px.choropleth(state_plot_final, 
                        locationmode = "USA-states",
                        locations = state_plot_final["Code"],
                        color = state_plot_final[field],
                        color_continuous_scale = "Sunset",
                        scope = "usa",
                       labels = drop_down_second)
    fig.layout.template = 'simple_white'
    fig.layout.title = {'text': 'Distribution of number of Doctorates Awarded by <br>' +
                       drop_down_second[field] + '<br>' + 
                        'in the US',
                       'y': 0.9,
                       'x': 0.5,
                       'xanchor': 'center',
                       'yanchor': 'middle'}
    fig.layout.paper_bgcolor = 'rgb(0,0,0,0)'
    fig.layout.plot_bgcolor = 'rgb(0,0,0,0)'
    fig.layout.height = 600
    fig.layout.width = 1000
    return fig.to_dict()


# the bar plot on the right side
@app.callback(Output('bar_education', 'figure'),
              [Input('year_slider', 'value'),
               Input('field_dropdown_top', 'value')])
def plot_bar_education(year, field):
    if not field:
        raise PreventUpdate
    df2 = year_field_plot.query('Year == @year')
    fig = go.Figure()
    fig.add_bar(x = df2["Category"],
                y = df2[field],
                orientation = 'v',
                marker = {'color': ['rgb(248,156,116)']*10})
    fig.layout.title = {'text': 'Number of Doctorates Awarded by <br>' +
                        drop_down_trans[field] + ' in ' +
                        str(year) + ' by Categories',
                        'x': 0.5,
                        'y': 0.9,
                        'xanchor': 'center',
                        'yanchor': 'middle'}
    fig.layout.template = 'simple_white'
    fig.layout.margin = {'l': 200}
    fig.layout.height = 600
    fig.layout.width = 600
    fig.layout.yaxis.showgrid = True
    fig.layout.paper_bgcolor = 'rgb(0,0,0,0)'
    fig.layout.plot_bgcolor = 'rgb(0,0,0,0)'
    return fig.to_dict()

# the bar plot on the left side
@app.callback(Output('bar_gender', 'figure'),
              [Input('year_slider', 'value'),
               Input('field_dropdown_top', 'value')])
def plot_bar_gender(year, field):
    if not field:
        raise PreventUpdate
    df3 = gender_plot.query('Year == @year')
    fig = go.Figure()
    fig.add_bar(x = df3["Gender"],
                y = df3[field],
                orientation = 'v',
                marker = {'color': ['rgb(220,176,242)']*10})
    fig.layout.title = {'text':'Number of Doctorates Awarded by <br>' +
                        drop_down_trans[field] + ' in ' +
                        str(year) + ' by Gender',
                        'x': 0.5,
                        'y': 0.9,
                        'xanchor': 'center',
                        'yanchor': 'middle'}
    fig.layout.template = 'simple_white'
    fig.layout.height = 570
    fig.layout.width = 450
    fig.layout.margin = {'l': 20}
    fig.layout.yaxis.showgrid = True
    fig.layout.paper_bgcolor = 'rgb(0,0,0,0)'
    fig.layout.plot_bgcolor = 'rgb(0,0,0,0)'
    return fig.to_dict()


if __name__ == '__main__':
    app.run_server()

/Users/ahoo/opt/anaconda3/lib/python3.8/site-packages/dash_bootstrap_components/_table.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Oct/2021 17:48:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2021 17:48:14] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2021 17:48:14] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2021 17:48:14] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2021 17:48:14] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2021 17:48:14] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2021 17:48:14] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2021 17:48:15] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2021 17:48:15] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2021 17:48:15] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2021 17:50:37] "POST /_dash-